<a href="https://colab.research.google.com/github/olanigan/yoruba_asr/blob/main/W2V2_Bert_Yor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
%%capture
!pip install datasets torchaudio jiwer -q
!pip install --upgrade transformers -q
!pip install accelerate -U -q


In [2]:
from huggingface_hub import notebook_login

notebook_login()


# Create Wav2Vec2CTCTokenizer

In [5]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("mozilla-foundation/common_voice_16_0", "yo", split="train+validation", use_auth_token=True)
common_voice_test = load_dataset("mozilla-foundation/common_voice_16_0", "yo", split="test", use_auth_token=True)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:2483: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_16_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_16_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1212it [00:00, 28928.05it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 863it [00:00, 63552.29it/s]


...


Reading metadata...: 998it [00:00, 62268.17it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1169it [00:00, 64349.91it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 229it [00:00, 84684.85it/s]
/usr/local/lib/python3.10/dist-packages/datasets/load.py:2483: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_16_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_16_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [6]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])


In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)


,sentence,variant
0,Ọkọ̀ olówó iyebíye mẹ́wàá nìkan ni ọlọ́lá yìí gùn nígbà ayé rẹ̀.,
1,Awọ fẹ́lẹ́ bonú ni kò jẹ́ ka rí kùn òṣìkà.,
2,Alága àjọ ìgbógun ti ògùn olóró ti gba ìwé ẹ̀rí lọ́wọ́ ààrẹ,
3,Èèyàn mẹ́tàdínlógún kú nínú ìjàmbá ọkọ̀ bọ́ọ̀sì.,
4,Kí ló fa wàhálà láàrin ọlọ́pàá àtàwọn ọlọ́kadà?,
5,Gbajúmọ̀ olukọ ni Fẹmi Ọlọ́runníṣọla lásìkò ogun jálumi,
6,Àwọn ajínigbé ti yìnbọn pa ara wọn lẹ́yìn èdè àiyedè lórí owó pínpín,
7,Àkóbá ni ò ń ṣe fún ìlera ọkàn rẹ tí ó bá sùn dada.,
8,Ojojúmọ́ ni ọ̀rọ̀ àwọn agbófinró yìí máa ń sú mi,
9,Ajá bu olè náà jẹ lápá.,


In [8]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\»\«]'

def remove_special_characters(batch):
    # remove special characters
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()

    return batch

common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)


Map:   0%|          | 0/2075 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

In [9]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))


,sentence,variant
0,obìnrin kògbérégbè nínú òṣèré yorùbá ni wọ́n ṣàfihàn lórí ètò,
1,ọmọ ilé ẹ̀kọ́ wa méjì fórí kó ọta ìbọn lọ́jọ́ kan ṣoṣo,
2,ọgọ́rùn ún èèyàn ló pàdánù ẹ̀mí wọn nínú ìkọlù tí ó sẹlẹ̀ ní èkìtì,
3,ilé ẹjọ́ ju àwọn ọmọ burúkú náà sẹ́wọ̀n,
4,àwọn akẹ́kọ̀ọ́ mẹ́rìnlà ló pàdánù ẹ̀mí wọn nínú ìjàm̀bá,
5,àwọn ọba ilẹ̀ yorùbá ní ìpàdé lọ́jọ́ ìṣẹ́gun,
6,ta ló buwọ́lu ìwé béèlì àwọn méjìlá tí ìjọba mú lọ sí ilé ẹjọ́,
7,lẹ́yìn tí mo borí ìdíje oníléjilé yìí mo gba owó púpọ̀,
8,èèyàn mọ́kànlá dèrò ọ̀run nibadan torí àrùn àìmọ̀dí,
9,kò sí amúgbálẹ́gbẹ̀ẹ́ inú ìtàn náà tí kò ní ipa tí ó ń kó,


In [10]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)


Map:   0%|          | 0/2075 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

In [11]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict


{' ': 0,
 'a': 1,
 'b': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'r': 16,
 's': 17,
 't': 18,
 'u': 19,
 'w': 20,
 'y': 21,
 'à': 22,
 'á': 23,
 'è': 24,
 'é': 25,
 'ì': 26,
 'í': 27,
 'ò': 28,
 'ó': 29,
 'ù': 30,
 'ú': 31,
 'ń': 32,
 'ǹ': 33,
 '̀': 34,
 '́': 35,
 '̄': 36,
 '̣': 37,
 'ṣ': 38,
 'ẹ': 39,
 'ọ': 40,
 'ụ': 41,
 '’': 42}

In [12]:
# prompt: Display redundant characters in vocab_dict

# Create a set of all unique characters in the vocabulary
unique_chars = set(vocab_dict.keys())

# Create a set of redundant characters
redundant_chars = set()

# Iterate through the vocabulary and find characters that appear more than once
for char in unique_chars:
  count = 0
  for key, value in vocab_dict.items():
    if char == key:
      count += 1
  if count > 1:
    redundant_chars.add(char)

# Display the redundant characters
print("Redundant characters in vocab_dict:")
for char in redundant_chars:
  print(char)


Redundant characters in vocab_dict:


In [ ]:
%%writefile cleanup.py3
def remove_latin_characters(batch):
    batch["sentence"] = re.sub(r'[a-z]+', '', batch["sentence"])
    return batch

# remove latin characters
common_voice_train = common_voice_train.map(remove_latin_characters)
common_voice_test = common_voice_test.map(remove_latin_characters)

# extract unique characters again
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict


In [13]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
len(vocab_dict)


43

In [14]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)


45

In [15]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)


In [16]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


## HF Tokenizer push

In [23]:
print(tokenizer)

Wav2Vec2CTCTokenizer(name_or_path='./', vocab_size=45, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	43: AddedToken("[UNK]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	44: AddedToken("[PAD]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	45: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [24]:
if isinstance(tokenizer, Wav2Vec2CTCTokenizer):
    print("Tokenizer is a Wav2Vec2CTCTokenizer")
else:
    print("Tokenizer is not a Wav2Vec2CTCTokenizer")

Tokenizer is a Wav2Vec2CTCTokenizer


In [18]:
import re
repo_name = "w2v-bert-yoruba-CV16"


None
Repository name is invalid


In [41]:
!huggingface-cli repo create $repo_name

git version 2.34.1
git-lfs/3.0.2 (GitHub; linux amd64; go 1.18.1)

You are about to create olanigan/w2v-bert-yoruba-CV16
Proceed? [Y/n] Y

Your repo now lives at:
  https://huggingface.co/olanigan/w2v-bert-yoruba-CV16

You can clone it locally with the command below, and commit/push as usual.

  git clone https://huggingface.co/olanigan/w2v-bert-yoruba-CV16



In [19]:

tokenizer.push_to_hub(repo_name)


CommitInfo(commit_url='https://huggingface.co/olanigan/w2v-bert-yoruba-CV16/commit/65da84974fda1ef454a4ddfad6b4aa122114c254', commit_message='Upload tokenizer', commit_description='', oid='65da84974fda1ef454a4ddfad6b4aa122114c254', pr_url=None, pr_revision=None, pr_num=None)